<a href="https://colab.research.google.com/github/dbamman/nlp21/blob/master/HW5/HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 5: Sequence Labeling

In this homework, you will be using a bigram hidden Markov model (HMM) for sequence labelling. We explore two decoding algorithms: greedy (already implemented) and Viterbi (you will be implementing this).

In [79]:
import numpy as np

In [80]:
# Get data
!wget https://raw.githubusercontent.com/dbamman/nlp21/main/HW5/pos.train

--2021-03-08 04:51:23--  https://raw.githubusercontent.com/dbamman/nlp21/main/HW5/pos.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1717454 (1.6M) [text/plain]
Saving to: ‘pos.train.2’

pos.train.2         100%[===================>]   1.64M  --.-KB/s    in 0.07s   

2021-03-08 04:51:24 (23.6 MB/s) - ‘pos.train.2’ saved [1717454/1717454]



In [81]:
def read_data(filename):
    """
    Helper function to read in data from a file.
    Outputs data points (X) and labels (Y), in two arrays of the same length.
    """
    sentences = []
    tags = []

    current_sentence = []
    current_tags = []

    with open(filename, encoding='utf8') as f:
        for line in f:
            if len(line) == 0:
                continue
            if line == '\n':
                if len(current_sentence) != 0:
                    sentences.append(current_sentence)
                    tags.append(current_tags)

                current_sentence = []
                current_tags = []
            else:
                columns = line.rstrip().split('\t')
                word = columns[0].lower()
                tag = columns[1]

                current_sentence.append(word)
                current_tags.append(tag)

        return sentences, tags

In [82]:
def create_vocab(train_x, train_y):
    """
    Outputs two dictionaries:
    (1) mapping each vocab word to an index of 0 thru LEN(TRAIN_X)-1
    (2) mapping each tag to an index of 0 thru LEN(TRAIN_Y)-1
    """
    x_vocab = {}
    y_vocab = {}
    for xs, ys in zip(train_x, train_y):
        for x, y in zip(xs, ys):
            if x not in x_vocab:
                x_vocab[x] = len(x_vocab)
            if y not in y_vocab:
                y_vocab[y] = len(y_vocab)

    y_vocab["START"] = len(y_vocab)
    y_vocab["END"] = len(y_vocab)

    return x_vocab, y_vocab

In [83]:
def estimate_parameters(x_vocab, y_vocab, train_x, train_y):
    """
    Returns a transitions matrix and emissions matrix with probabilities
    given in LOG space.
    transitions[s1, s2]: log probability of observing label s2 after label s1.
    emissions[s, t]: log probability of observing token t with label s.
    """
    transitions = np.zeros((len(y_vocab)-1, len(y_vocab)))
    emissions = np.zeros((len(y_vocab)-2, len(x_vocab)))

    for xs, ys in zip(train_x, train_y):
        prev = y_vocab["START"]
        for i in range(len(ys)):
            y_idx = y_vocab[ys[i]]
            x_idx = x_vocab[xs[i]]
            transitions[prev][y_idx] += 1
            prev = y_idx
            emissions[y_idx][x_idx] += 1
        transitions[prev][y_vocab["END"]] += 1

    # normalize each row by its total
    transitions = transitions/np.sum(transitions, axis=1, keepdims=True)
    emissions = emissions/np.sum(emissions, axis=1, keepdims=True)

    # let's work in log space (adding instead of multiplying)
    transitions = np.log(transitions)
    emissions = np.log(emissions)

    return transitions, emissions

In [84]:
train_x, train_y = read_data("pos.train")

x_vocab, y_vocab = create_vocab(train_x, train_y)
transitions, emissions = estimate_parameters(x_vocab, y_vocab, train_x, train_y)
# You might see a "RuntimeWarning: divide by zero encountered in log" error — that's ok!

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


Run the following cell to understand the sizes and dimensions of the vocabulary, POS tag labels, transitions, and emissions.

In [85]:
print("# of words in vocabulary: %d" % len(x_vocab))
print("# of POS tag labels (including START and END): %d" % len(y_vocab))
print("dimension of transitions: %s" % (transitions.shape,))
print("dimension of emissions: %s" % (emissions.shape,))

# of words in vocabulary: 16650
# of POS tag labels (including START and END): 52
dimension of transitions: (51, 52)
dimension of emissions: (50, 16650)


The "START" and "END" tags are considered "non-emitting" states and therefore have no corresponding entries in the emissions matrix. (In other words, there's no probability of observing a word token labeld "START" or "END" since those are special labels.) According to the `y_vocab` map, `START` and `END` correspond to indices 50 and 51 respectively.

## Greedy Decode
This decoding algorithm proceeds left to right, committing to the best tag for each time step (given the previous tag label and current time step observation). You do not need to modify anything in the `greedy_decode` function.

In [86]:
def greedy_decode(transitions, emissions, y_vocab, x_vocab, sequence):
    prev = y_vocab["START"]
    best_prediction = []
    for x in sequence:
        maxlogprob = -100000000
        best_y = None
        for y in y_vocab:
            if y != "END" and y != "START":
                logprob = transitions[prev][y_vocab[y]] + emissions[y_vocab[y]][x_vocab[x]]
                if logprob > maxlogprob:
                    maxlogprob = logprob
                    best_y = y
        prev = y_vocab[best_y]
        best_prediction.append(best_y)

    return ' '.join(best_prediction)

The following cell runs the `greedy_decode` algorithm on the sentence "fruit flies like a banana". Take a look at the sequence labels it predicts.

In [87]:
greedy_prediction = greedy_decode(transitions, emissions, y_vocab, x_vocab, ["fruit", "flies", "like", "a", "banana"])
print(greedy_prediction)

NN VBZ IN DT NN


## Viterbi Decode
Your task is to implement the Viterbi algorithm to find the optimal sequence of POS tag labels for a sentence or word sequence.

In [97]:
def viterbi_decode(transitions, emissions, y_vocab, x_vocab, sequence):
    """
    transitions : a matrix where the entry transitions[s1, s2] is the log
                  probability of observing label s2 after label s1 in a sequence.
    emissions   : a matrix where the entry emissions[s, t] is the log 
                  probability of observing token t with label s.
    y_vocab     : a dictionary mapping each POS tag label to an index of 0 thru
                  the number of POS tag labels.
    x_vocab     : a dictionary mapping each vocab word to an index of 0 thru the
                  number of vocab words.
    sequence    : a list of (string) words/tokens.

    Returns a string with the POS tag labels (separated by spaces) for the words
    in the sequence.
    For example,

    >>> viterbi_decode(transitions, emissions, y_vocab, x_vocab, ["fruit", "flies", "like", "a", "banana"])
    "NN VBZ IN DT NN"

    Do not change the order or naming of the input parameters.
    """

    # YOUR CODE HERE
    N = len(y_vocab) # num of POS labels, including START and END
    T = len(sequence) # num of words in sequence

    # initiate the probability and backpointer to be -inf for all cells
    # column index 0 for START, followed T columns for words in sequence
    viterbi = np.ones((N, T+1)) * -np.inf
    backpointer = np.ones((N, T+1)) * -np.inf

    # initiation
    prev =  y_vocab["START"] # index of initial POS tag
    viterbi[:, 0] = transitions[prev]
    backpointer[:, 0] = 0

    for t, x in enumerate(sequence): # t as index from 0 to T
      for y in y_vocab:
        if y != "END" and y != "START":
          viterbi[y_vocab[y], t+1] = np.max(viterbi[:-1, t] + transitions[:, y_vocab[y]] + emissions[y_vocab[y], x_vocab[x]])
          backpointer[y_vocab[y], t+1] = np.argmax(viterbi[:-1, t] + transitions[:, y_vocab[y]] + emissions[y_vocab[y], x_vocab[x]])

    # backtrack the best labels for the sequence, bestpath record index from T to 1
    bestpath = []
    lastbestindex = np.argmax(viterbi[:, T])
    bestpath.append(lastbestindex)

    i = lastbestindex
    while T >= 2:
      i = int(backpointer[i, T])
      bestpath.append(i)
      T -= 1

    bestpath = bestpath[::-1] # reverse the list

    # replace y_vocab row index with respective y label
    for i in range(len(bestpath)):
      for y, row_idx in y_vocab.items():
        if row_idx == bestpath[i]:
          bestpath[i] = y
    
    return " ".join(bestpath)

In [98]:
viterbi_prediction = viterbi_decode(transitions, emissions, y_vocab, x_vocab, ["fruit", "flies", "like", "a", "banana"])
print(viterbi_prediction)

NN VBZ IN DT NN


The following cell runs the `viterbi_decode` algorithm on the sentence "fruit flies like a banana". For this particular sequence, the Viterbi-predicted sequence labels should be the same as the ones predicted by the greedy decode.

## Examples: Greedy vs Viterbi

In [99]:
def create_compare_fn(transitions, emissions, y_vocab, x_vocab):
    """
    Helper function to compare results from the greedy decode and viterbi decode.
    You do not have to modify this function.
    """
    def compare_greedy_and_viterbi(sentence):
        sequence = sentence.lower().split(" ")
        greedy_pred = greedy_decode(transitions, emissions, y_vocab, x_vocab, sequence)
        viterbi_pred = viterbi_decode(transitions, emissions, y_vocab, x_vocab, sequence)
        return greedy_pred, viterbi_pred
    return compare_greedy_and_viterbi

compare_greedy_and_viterbi = create_compare_fn(transitions, emissions, y_vocab, x_vocab)

The following cells run the greedy algorithm and Viterbi algorithm on various sentences. Take a look at the sequence labels it predicts for both. If your Viterbi implementation is correct, the greedy and Viterbi sequence labels should be different for all of the following examples.

In [100]:
g, v = compare_greedy_and_viterbi("Bear the feeling .")
print(g)
print(v)

NN DT NN .
VB DT NN .


In [101]:
g, v = compare_greedy_and_viterbi("Present an object .")
print(g)
print(v)

JJ DT NN .
VBP DT NN .


In [102]:
g, v = compare_greedy_and_viterbi("The judge objects to the suspect speaking .")
print(g)
print(v)

DT NN VBZ TO RB VBP VBG .
DT NN VBZ IN DT NN VBG .


Let's look at a garden path sentence (similar to the one from class, though "raced" is not in our vocabulary in this dataset). 

In [103]:
g, v = compare_greedy_and_viterbi("The horse led past the barn fell .")
print(g)
print(v)

DT NN VBD JJ DT NN VBD .
DT NN VBN IN DT NN VBD .
